<center><h1>Tang_Lin_HW3</h1></center>
<br>
<br>

Name: Tang Lin
<br>
Github Username: MeLinOrDoreen
<br>
USC ID: 9012859074

## 1. Time Series Classification Part 1: Feature Creation/Extraction

### (a) Download Data

Package imports

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import os
import random
from scipy.stats import bootstrap

# to surpress warnings
import warnings
warnings.filterwarnings("ignore")

Get the AReM Data Set

In [10]:
# Define a function to help us load the data sets
data_path = '../Data/AReM/' # we have already download the AReM data set in our local environment

### (b) Test and Train Data

In [11]:
## Define a function to import multiple data sets in to one dataframe
def load_data(folder_path, file_range, test = True):
    data = pd.DataFrame()
    file_names = sorted(os.listdir(folder_path)) # Get all the file names in one folder

    if test == True:
        file_names = file_names[0:file_range] # According to the setting, to get the first few data sets as test data sets
    else:
        file_names = file_names[file_range:] # Choose the rest few data sets as train data sets

    for i, file_name in enumerate(file_names):
        file_path = os.path.join(folder_path, file_name)
        df = pd.read_csv(file_path, skiprows= 4)
        df = pd.DataFrame(df)
        data = pd.concat([data,df])
        data = pd.DataFrame(data)

    return data


In [12]:
# Test data set
test_data = load_data("../Data/AReM/bending1", 2, test = True)
test_data = pd.concat([test_data,load_data("../Data/AReM/bending2", 2, test = True )])
test_data = pd.concat([test_data,load_data("../Data/AReM/cycling", 3, test = True )])
test_data = pd.concat([test_data,load_data("../Data/AReM/lying", 3, test = True )])
test_data = pd.concat([test_data,load_data("../Data/AReM/sitting", 3, test = True )])
test_data = pd.concat([test_data,load_data("../Data/AReM/standing", 3, test = True )])
test_data = pd.concat([test_data,load_data("../Data/AReM/walking/", 3, test = True )])

test_data

,# Columns: time,avg_rss12,var_rss12,avg_rss13,var_rss13,avg_rss23,var_rss23
0,0,39.25,0.43,22.75,0.43,33.75,1.30
1,250,39.25,0.43,23.00,0.00,33.00,0.00
2,500,39.25,0.43,23.25,0.43,33.00,0.00
3,750,39.50,0.50,23.00,0.71,33.00,0.00
4,1000,39.50,0.50,24.00,0.00,33.00,0.00
...,...,...,...,...,...,...,...
475,118750,36.50,2.87,18.50,2.29,16.00,2.16
476,119000,35.00,9.70,18.67,0.94,18.33,2.05
477,119250,28.25,4.82,16.00,3.27,22.00,5.66
478,119500,32.25,8.26,12.50,4.15,15.25,2.77


In [13]:
# train data set
train_data = load_data("../Data/AReM/bending1", 2, test = False)
train_data = pd.concat([train_data,load_data("../Data/AReM/bending2", 2, test = False )])
train_data = pd.concat([train_data,load_data("../Data/AReM/cycling", 3, test = False )])
train_data = pd.concat([train_data,load_data("../Data/AReM/lying", 3, test = False )])
train_data = pd.concat([train_data,load_data("../Data/AReM/sitting", 3, test = False )])
train_data = pd.concat([train_data,load_data("../Data/AReM/standing", 3, test = False )])
train_data = pd.concat([train_data,load_data("../Data/AReM/walking/", 3, test = False )])

train_data


,# Columns: time,avg_rss12,var_rss12,avg_rss13,var_rss13,avg_rss23,var_rss23
0,0,42.00,0.71,21.25,0.43,30.00,0.00
1,250,41.50,0.50,20.25,1.48,31.25,1.09
2,500,41.50,0.50,14.25,1.92,33.00,0.00
3,750,40.75,0.83,15.75,0.43,33.00,0.00
4,1000,40.00,0.71,20.00,2.74,32.75,0.43
...,...,...,...,...,...,...,...
475,118750,31.50,1.66,12.50,3.20,14.25,4.44
476,119000,27.33,1.25,11.33,0.94,20.00,4.00
477,119250,37.80,7.68,14.20,2.48,17.25,0.83
478,119500,33.75,1.30,15.75,5.21,16.50,2.69


In [14]:
# Check the shape of train data set and test data set
print(test_data.shape)
print(train_data.shape)

(9120, 7)
(33119, 7)


### (c) Feature Extraction

#### i. Research


- Basic Statistical Feature

1. Mean: The average value of the time series.

2. Minimum: The smallest value in the time series.

3. Maximum: The largest value in the time series.

4. Range: The difference between the maximum and minimum values.

5. Variance: Measures the spread or dispersion of the values in the time series.

6. Standard Deviation: Indicates how much the values deviate from the mean.

7. Skewness: Measures the asymmetry of the distribution of the values.

8. Kurtosis: Measures the "tailedness" or shape of the distribution of the values.

9. Root Mean Square (RMS): The square root of the average of the squared values.

10. Zero Crossing Rate: The rate at which the signal changes from positive to negative or vice versa.

11. Mean Absolute Deviation (MAD): The average of the absolute differences between each data point and the mean.

12. Interquartile Range (IQR): The range between the first quartile (25th percentile) and the third quartile (75th percentile).

13. Percentiles: Values below which a given percentage of observations fall (e.g., 25th percentile, 50th percentile (median), 75th percentile).

<br>
<br>

- Time Series Statistics Features

1. Entropy: A measure of disorder or unpredictability in the time series.

2. Autocorrelation: Measures the correlation of the time series with a lagged version of itself.

3. Cross-Correlation: Measures the similarity between two different time series.

4. Lag Features: Values from previous time steps used as features.

5. Time-domain Signal Characteristics (e.g., number of peaks, zero crossings, etc.): Various characteristics related to the shape of the time series.


#### ii. Extraction

In [16]:
# Combine the train and test datasets 
full_data = pd.concat([test_data, train_data]).iloc[:, 1:] # only need the 6 time series columns
full_data["Instance"] = full_data.index

# Divide into difference instances
full_data = full_data.reset_index()

for i in range(1, 89):
    full_data.loc[(i-1)*480 : i*480 - 1, "Instance"] = i

# Check if instances have same number of rows: instance 88 has only 479 rows, but it won't cause troubles for our analysis
for i in range(1, 88):
    if (full_data["Instance"] == i).sum() != (full_data["Instance"] == i +1 ).sum():
        print(f"Different Length: Instance {i} & Instance {i+1}")

# Get the modified full data frame
full_data = full_data.iloc[:, 1:]

Different Length: Instance 87 & Instance 88


In [17]:
# Get the summaried features
feature_summary = full_data.groupby("Instance").describe() 

# Rename the column names
feature_summary.columns = ['_'.join(col).strip() for col in feature_summary.columns.values] 

# Drop the features "counts"
feature_summary = feature_summary.drop(columns = feature_summary.filter(like='count').columns)

feature_summary

,avg_rss12_mean,avg_rss12_std,avg_rss12_min,avg_rss12_25%,avg_rss12_50%,avg_rss12_75%,avg_rss12_max,var_rss12_mean,var_rss12_std,var_rss12_min,...,avg_rss23_50%,avg_rss23_75%,avg_rss23_max,var_rss23_mean,var_rss23_std,var_rss23_min,var_rss23_25%,var_rss23_50%,var_rss23_75%,var_rss23_max
Instance,,,,,,,,,,,,,,,,,,,,,
1,40.624792,1.476967,37.25,39.2500,40.500,42.00,45.00,0.358604,0.322605,0.0,...,35.00,36.00,38.25,0.570583,0.582915,0.0,0.00,0.43,1.300,1.92
2,42.812812,1.435550,38.00,42.0000,42.500,43.67,45.67,0.372437,0.289158,0.0,...,33.00,34.50,38.50,0.571083,0.601010,0.0,0.00,0.43,1.300,3.11
3,24.562958,3.737514,12.75,23.1875,24.250,26.50,51.00,0.590833,0.837408,0.0,...,23.75,27.00,30.00,0.700188,0.693720,0.0,0.43,0.50,0.870,4.97
4,27.464604,3.583582,0.00,25.5000,28.000,30.00,42.75,0.449708,0.767197,0.0,...,18.00,20.75,33.00,1.122125,1.012342,0.0,0.47,0.83,1.300,6.76
5,37.177042,3.581301,24.25,34.5000,36.250,40.25,45.00,2.374208,1.601799,0.0,...,20.00,21.75,25.50,2.921729,1.852600,0.0,1.50,2.50,3.900,9.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,34.770104,4.745938,20.75,31.6700,35.290,38.25,46.25,4.231521,2.279355,0.0,...,16.29,18.33,25.50,3.290875,1.643302,0.0,2.05,3.27,4.305,9.39
85,34.921750,4.643832,21.50,32.0000,35.500,38.00,51.00,4.109062,2.333053,0.0,...,16.25,18.25,25.00,3.285792,1.698713,0.0,2.12,3.03,4.500,10.21
86,34.316375,4.969229,18.33,31.2500,34.750,38.00,47.67,4.415938,2.593868,0.0,...,16.00,18.00,24.00,3.262583,1.617908,0.0,2.05,2.98,4.320,8.01


#### iii. Standard Deviation

In [20]:
# Get the standard deviation for each extracted feature
sd_summary = feature_summary.agg("std")

sd_df = pd.DataFrame({'Standar_Deviation': sd_summary})

sd_df

,Standar_Deviation
avg_rss12_mean,5.335265
avg_rss12_std,1.770286
avg_rss12_min,9.537988
avg_rss12_25%,6.151927
avg_rss12_50%,5.440710
avg_rss12_75%,5.139367
avg_rss12_max,4.394362
var_rss12_mean,1.574357
var_rss12_std,0.884015
var_rss12_min,0.000000


In [19]:
# Use the bootstrap to get the confidence interaval for standard deviation of each feature
bootstrap_ci = bootstrap((feature_summary.values,), np.std, confidence_level=0.90,
                         random_state=1, method='percentile')

CI = bootstrap_ci.confidence_interval

sd_df["Low_CI_Bootstrap"] = CI[0]

sd_df["High_CI_Bootstrap"] = CI[1]

sd_df

,Standar_Deviation,Low_CI_Bootstrap,High_CI_Bootstrap
avg_rss12_mean,5.335265,4.675494,5.847353
avg_rss12_std,1.770286,1.557181,1.940303
avg_rss12_min,9.537988,8.200076,10.700431
avg_rss12_25%,6.151927,5.541748,6.608118
avg_rss12_50%,5.440710,4.769074,5.970626
avg_rss12_75%,5.139367,4.292977,5.820846
avg_rss12_max,4.394362,3.306386,5.262762
var_rss12_mean,1.574357,1.391348,1.698146
var_rss12_std,0.884015,0.797840,0.936563
var_rss12_min,0.000000,0.000000,0.000000


#### iv. Select Features

- Mean: for this indicates central tendency and it is widely used to calculate other statistics, e.g. moving average, correlation. And since this is a time series data, it might be useful to calculate moving average and autocorrelation.
- Median: it is robust to outliers, and gives a measure of the middle value of the dataset and is especially useful when dealing with skewed distributions.
- Standard deviation: it measures the dispersion or spread of a dataset.
- Combine all these 3 features, we can get a comprehensive understanding of the data set (variance, skewness...)

## 2. ISLR 3.7.4

### (a) Linear Train

We will expect the cubic regression to have lower RSS. Because the cubic regression have more predictors and is more flexible than te linear regression. We will usually expect regression with more predictors and flexibilities to fit the relationship better. 

### (b) Linear Test

More flexibilities/predictors may lead to the problem of overfitting, so we will expect the cubic regression to have higher test RSS.

### (c) Not Linear Train

If the true relationship is not linear, the cubic regression might offer lower training RSS for it can capture more nonliear relationship. 

### (d) Not Linear Testing

We will expect the cubic regression to have lower test RSS, because the cubic regression can better capture the true nonliear relationshio.

## 3. ISLR 3.7.3 - Extra Practice 

### (a) 
`Y = 50 + 20 * GPA + 0.07 * IQ + 35 * Level + 0.01 * GPA : IQ - 10 * GPA : Level`

iii is correct 
<br>
f(..., Level = 1) - f(..., level - 0) = 35 - 10 * GPA
<br>
35 - 10 * GPA > 0, GPA < 3.5
<br>
So if GPA is High enough, the high school graduates earn more on average.

### (b)

Level = 1, IQ = 110, GPA = 4.0

In [ ]:
50 + 20 * 4 + 0.07 * 110 + 35 * 1 + 0.01 * 4 * 110 - 10 * 4 *1

137.1

### (c)

False
<br>
The coefficenit value doesn't give enough info about if the interaction is statistically significant. We should examine the p-value.

## 3. ISLR 3.7.5 - Extra Practice 

$$ \hat{y}_{k} = x_{k} \times \frac{\sum_{i=1}^{n}\left ( x_{i} y_{i} \right )}{\sum_{i'=1}^{n} x_{i'}^{2}} $$


$$ \hat{y}_{k} = \sum_{i =1}^{n} \frac{\left ( x_{k} x_{i} \right ) \times y_{i}}{\sum_{i'=1}^{n} x_{i'}^{2}} $$

$$ a_{i'} = \frac{ x_{k} x_{i} } { \sum_{i'=1}^{n} x_{i'}^{2} } $$